In [1]:
from ray import serve

from io import BytesIO
from PIL import Image
from starlette.requests import Request
from typing import Dict

import torch
from torchvision import transforms
from torchvision.models import resnet18

In [2]:
@serve.deployment
class ImageModel:
    def __init__(self):
        self.model = resnet18(pretrained=True).eval()
        self.preprocessor = transforms.Compose(
            [
                transforms.Resize(224),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Lambda(lambda t: t[:3, ...]),  # remove alpha channel
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    async def __call__(self, starlette_request: Request) -> Dict:
        image_payload_bytes = await starlette_request.body()
        pil_image = Image.open(BytesIO(image_payload_bytes))
        print("[1/3] Parsed image data: {}".format(pil_image))

        pil_images = [pil_image]  # Our current batch size is one
        input_tensor = torch.cat(
            [self.preprocessor(i).unsqueeze(0) for i in pil_images]
        )
        print("[2/3] Images transformed, tensor shape {}".format(input_tensor.shape))

        with torch.no_grad():
            output_tensor = self.model(input_tensor)
        print("[3/3] Inference done!")
        return {"class_index": int(torch.argmax(output_tensor[0]))}

In [3]:
image_model = ImageModel.bind()


In [7]:
serve.run(image_model)


2024-06-25 11:44:22,378	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(ProxyActor pid=1566120) INFO 2024-06-25 11:44:25,161 proxy 10.138.187.90 proxy.py:1165 - Proxy starting on node 8ffd64913c370a1cec0d90bd6939baf7cf8d143d1b562cde1e75f413 (HTTP port: 8000).
2024-06-25 11:44:25,198	INFO handle.py:126 -- Created DeploymentHandle 'lhws518b' for Deployment(name='ImageModel', app='default').
2024-06-25 11:44:25,199	INFO handle.py:126 -- Created DeploymentHandle 'toio05p2' for Deployment(name='ImageModel', app='default').
(ServeController pid=1566029) INFO 2024-06-25 11:44:25,272 controller 1566029 deployment_state.py:1598 - Deploying new version of Deployment(name='ImageModel', app='default') (initial target replicas: 1).
(ServeController pid=1566029) INFO 2024-06-25 11:44:25,375 controller 1566029 deployment_state.py:1844 - Adding 1 replica to Deployment(name='ImageModel', app='default').
(ServeReplica:default:ImageModel pid=1566211) /home/akar

DeploymentHandle(deployment='ImageModel')

(ServeReplica:default:ImageModel pid=1566211) INFO 2024-06-25 11:45:03,152 default_ImageModel ie4nqt0g 5b2c6460-3a18-415e-8df2-4041bb4adcc5 / replica.py:373 - __CALL__ OK 284.4ms
(ServeReplica:default:ImageModel pid=1566211) ERROR 2024-06-25 11:45:57,569 default_ImageModel ie4nqt0g 33c90b70-0b5d-4d14-8373-b97c4397af6b / replica.py:359 - Request failed:
(ServeReplica:default:ImageModel pid=1566211) ray::ServeReplica:default:ImageModel.handle_request_with_rejection() (pid=1566211, ip=10.138.187.90)
(ServeReplica:default:ImageModel pid=1566211)   File "/home/akarx/MLOps/mlops_venv/lib/python3.10/site-packages/ray/serve/_private/utils.py", line 168, in wrap_to_ray_error
(ServeReplica:default:ImageModel pid=1566211)     raise exception
(ServeReplica:default:ImageModel pid=1566211)   File "/home/akarx/MLOps/mlops_venv/lib/python3.10/site-packages/ray/serve/_private/replica.py", line 1132, in call_user_method
(ServeReplica:default:ImageModel pid=1566211)     await self._call_func_or_gen(
(Ser

(ServeReplica:default:ImageModel pid=1566211) INFO 2024-06-25 11:47:13,003 default_ImageModel ie4nqt0g 36d66e74-2c78-45c1-be1a-71a5584e1eea / replica.py:373 - __CALL__ OK 236.7ms
(ServeReplica:default:ImageModel pid=1566211) ERROR 2024-06-25 11:47:47,516 default_ImageModel ie4nqt0g b3dfa81a-ea0c-49f3-98e3-27c003a57c02 / replica.py:359 - Request failed:
(ServeReplica:default:ImageModel pid=1566211) ray::ServeReplica:default:ImageModel.handle_request_with_rejection() (pid=1566211, ip=10.138.187.90)
(ServeReplica:default:ImageModel pid=1566211)   File "/home/akarx/MLOps/mlops_venv/lib/python3.10/site-packages/ray/serve/_private/utils.py", line 168, in wrap_to_ray_error
(ServeReplica:default:ImageModel pid=1566211)     raise exception
(ServeReplica:default:ImageModel pid=1566211)   File "/home/akarx/MLOps/mlops_venv/lib/python3.10/site-packages/ray/serve/_private/replica.py", line 1132, in call_user_method
(ServeReplica:default:ImageModel pid=1566211)     await self._call_func_or_gen(
(Ser

In [9]:
import requests

ray_logo_bytes = requests.get(
    "https://raw.githubusercontent.com/ray-project/"
    "ray/master/doc/source/images/ray_header_logo.png"
).content

resp = requests.post("http://localhost:8000/", data=ray_logo_bytes)
print(resp.json())

(ServeReplica:default:ImageModel pid=1566211) [1/3] Parsed image data: <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=4726x1478 at 0x7EE75E6A8730>
(ServeReplica:default:ImageModel pid=1566211) [2/3] Images transformed, tensor shape torch.Size([1, 3, 224, 224])
(ServeReplica:default:ImageModel pid=1566211) [3/3] Inference done!


{'class_index': 919}
